##Test

In [10]:
import pandas as pd
import re
from sklearn.preprocessing import MultiLabelBinarizer

# 1. Đọc file RAW.csv và tách ingredients thành list
df = pd.read_csv('RAW.csv', engine='python', on_bad_lines='skip')
df['Ingredients_list'] = df['Ingredients'].str.split('|').apply(
    lambda lst: [item.strip() for item in lst] if isinstance(lst, list) else []
)

# 2. Các hàm hỗ trợ normalize và trích xuất tên cơ bản
def extract_quantity(raw):
    raw = raw.strip()
    match = re.match(r'^([\d]+(?:[\/\d]+)?|[½¼¾⅓⅔]+)', raw)
    return match.group(1) if match else ''

COMMON_KEYWORDS = {
    "garlic", "parsley","sardine","tarragon", "bacon", "olive oil", "mussel", "cider", "yoghurt", "bread", 
    "linguine", "chilli", "butter", "fennel seeds", "crabmeat", "asparagus", "lemon", "pea shoots", "milk", "yeast", "flour",
    "mixed spice", "sugar", "orange zest", "vegetable oil", "chocolate", "berries", "icing sugar", "peas", "potatoes",
    "chives", "salmon", "mint", "couscous", "steak", "almonds", "honey", "cherry tomatoes",
    "bream", "scallop", "prawn", "clam", "lemongrass", "ginger", "lime leaves", "coriander", "fish sauce",
    "soy sauce", "bok choi", "sugar snap peas", "broccoli", "choi sum", "rice", "cream", "onions",
    "red wine vinegar", "Brussels sprouts", "radishes", "herbs", "cranberries", "pastry", "mincemeat", "brandy", "apple", "apricots",
    "turmeric", "cumin seeds", "cauliflower", "paneer cheese", "groundnut oil", "curry leaves", "garam masala", "paprika",
    "plum tomatoes", "chickpeas", "coconut milk", "mango chutney", "carrots", "cucumber", "cabbage", "self-raising flour",
    "mustard seeds", "stone fruit", "strawberries", "brown sugar", "oats", "avocados", "bananas", "desiccated coconut", "mangos", "fennel", "leek",
    "celery", "thyme", "white fish", "white wine", "cashew nuts", "cocoa powder", "salt", "whole milk",
    "Rich Tea biscuits", "black olives", "squid", "penne", "artichoke", "sea salt", "buttermilk",
    "pepper", "broad beans", "spinach", "chicken stock", "prosciutto", "lamb shoulder", "oregano", "feta cheese", "pork shoulder",
    "flatbreads", "tomato", "white cabbage", "chicken breasts", "courgettes", "margarine",
    "Parmesan cheese", "courgette flowers", "bay leaves", "butternut squash",
    "ricotta cheese", "basil", "7-veg tomato sauce", "rocket", "acorn squash", "coriander seeds",
    "rosemary", "parsnips", "clementines", "celeriac", "sage leaves", "turnips", "nutmeg", "horseradish",
    "crème fraîche", "mushrooms", "Stilton cheese", "baby capers", "pecorino cheese", "spaghetti", "passata",
    "Prosecco", "clementine juice", "lime juice", "pomegranate juice", "passion fruit", "grapefruit juice", "elderflower cordial",
    "cloves", "cinnamon stick", "vanilla pod", "star anise", "kale", "rose petals", "lamb mince", "beef mince", "pork mince", "rabbit mince", "chicken mince", "minced lamb",
    "black peppercorns", "cardamom pods", "venison", "dairy-free margarine", "plain flour", "dairy-free milk chocolate",
    "dark chocolate", "golden syrup", "gingernut biscuits", "peanut & raisin mix", "marshmallows",
    "asafoetida", "chervil", "vanilla bean paste", "blackberries", "cherries", "blueberries", "coconut flakes", "cannelloni tubes",
    "semi-skimmed milk", "pancetta", "salami", "meat stock", "Arborio risotto rice", "Barbera d'Asti wine", "borlotti beans", "sesame seeds", "watermelon", "sesame oil",
    "purple sprouting broccoli", "goat’s cheese", "spring onions", "jalapeños", "corn cobs",
    "cottage cheese", "polenta", "Cheddar cheese", "demerara sugar", "marmalade", "chicken", "pearl barley", "green beans",
    "aubergine", "marinated olives", "flowering oregano", "water chestnuts",
    "Shaoxing rice wine", "dumpling flour", "black rice vinegar", "chilli oil", "gluten-free sausages",
    "palm sugar", "vinegar", "puff pastry", "lettuce", "red pepper", "basmati rice",
    "unsalted peanuts", "golden caster sugar", "gluten-free self-raising flour", "lamb neck",
    "orange", "fresh fruit", "corn tortilla wraps", "guacamole",
    "chunky tomato salsa", "natural yogurt", "duck leg", "chipotle paste", "mini tortillas", "chickpea flour", "ground cumin", "broccoletti",
    "higher-welfare sausage", "anchovy fillets", "Frascati white wine", "kimchi", "silken tofu", "stir-fry veg",
    "Manchego cheese", "arrabbiata sauce", "king prawns", "raw peas", "pork ribs", "baby back ribs",
    "Comté cheese", "Gruyère cheese", "wafer-thin ham", "beef short ribs", "dried marigold petals",
    "chorizo", "chestnuts", "sherry vinegar", "madeira sponge", "sweet sherry", "amaretti biscuits",
    "brambles", "egg yolks", "cornflour", "cockles", "granary bread", "vegetable or fish stock", "lamb neck",
    "dill", "lime", "white caster sugar", "glucose", "sweet potato", "peaches", "rabbit", "lamb cutlets",
    "frozen peas", "turkey", "short pasta", "smoked trout", "crème fraîche", "duck breast", "low-fat mayonnaise",
    "mixed salad leaves", "duck", "pheasant", "grouse", "partridge", "quail", "juniper berries", "bread flour", "shredded suet",
    "monkfish fillets", "vanilla essence", "orange blossom water", "gluten-free baking powder", "orange juice", "raspberries", "caster sugar", "Drambuie",
    "pork loin", "oyster sauce", "roasted peppers", "bean", "egg", "shallots", "baby leeks", "pastina", "hot-smoked trout",
    "rapeseed oil", "baharat spice mix", "tartare sauce", "sprouting broccoli", "dried apricots", "pumpkin seeds", "mango", "hearts of palm", "little gem lettuces", "tahini", "dried fruit", "raisins",
    "nuts", "mixed greens", "coconut oil", "fenugreek seeds", "fenugreek leaves", "rice vinegar", "plain wholemeal flour", "lemon zest", "mozzarella", "quinoa", "vanilla extract", "wine vinegar"
}

UNITS = {
    'g', 'kg', 'ml', 'l', 'tbsp', 'tsp', 'cup', 'cups', 'tablespoon',
    'tablespoons', 'teaspoon', 'teaspoons', 'bunch', 'clove', 'cloves',
    'slice', 'slices', 'pinch', 'handful', 'oz', 'ounce', 'ounces',
    'stick', 'sticks', 'dl', 'lb', 'lbs', 'dash', 'drop'
}

def normalize_ingredient(raw):
    s = raw.strip()
    if not s:
        return ''
    qty = extract_quantity(s)
    if qty:
        s = s[len(qty):].strip()
    s = s.replace('-', ' ')
    tokens = s.split()
    if tokens:
        first = tokens[0].lower().rstrip('.,')
        if first in UNITS:
            tokens = tokens[1:]
    joined = ' '.join(tokens)
    clean = re.sub(r'[\(\)\[\]\,\.\;]', '', joined)
    return clean.lower().strip()

def get_base_name(cleaned):
    tokens = cleaned.split()
    n = len(tokens)
    # kiểm tra bigram trước
    for i in range(n - 1):
        bigram = f"{tokens[i]} {tokens[i+1]}".lower()
        if bigram in COMMON_KEYWORDS:
            return bigram
    # nếu không có bigram, check từng token
    for token in tokens:
        t = token.lower()
        if t in COMMON_KEYWORDS:
            return t
        if t.endswith('ies'):
            singular = t[:-3] + 'y'
            if singular in COMMON_KEYWORDS:
                return singular
        if t.endswith('es'):
            singular = t[:-2]
            if singular in COMMON_KEYWORDS:
                return singular
        if t.endswith('s'):
            singular = t[:-1]
            if singular in COMMON_KEYWORDS:
                return singular
    # fallback: lấy token cuối cùng
    return tokens[-1].lower() if tokens else ''

# 3. Xây dựng danh sách tên ingredient cho mỗi dòng
ingredient_names_per_row = []
for ing_list in df['Ingredients_list']:
    names = set()
    for raw in ing_list:
        cleaned = normalize_ingredient(raw)
        if not cleaned:
            continue
        base = get_base_name(cleaned)
        if base:
            names.add(base)
    ingredient_names_per_row.append(list(names))

# 4. Tạo binary matrix với MultiLabelBinarizer
mlb = MultiLabelBinarizer()
binary_matrix = mlb.fit_transform(ingredient_names_per_row)
binary_df = pd.DataFrame(binary_matrix, columns=mlb.classes_, index=df.index)

pairs = [
    ("steak", "steaks"),
    ("peach", "peaches"),
    ("pie", "pies"),
    ("radish", "radishes"),
    ("potato", "potatoes"),
    ("dill","dilll"),
    ("sardine","sardines")
]

for singular, plural in pairs:
    if singular in binary_df.columns and plural in binary_df.columns:
        # Gộp: cột số ít = OR(cột số ít, cột số nhiều)
        binary_df[singular] = (binary_df[singular] | binary_df[plural]).astype(int)
        # Xóa cột số nhiều đi
        binary_df.drop(columns=[plural], inplace=True)

# 6. Tự động gộp tất cả các cặp số nhiều–số ít còn lại
cols = list(binary_df.columns)
for col in cols:
    if col.endswith("s"):
        singular = col[:-1]
        if singular in cols:
            # gộp 2 cột
            binary_df[singular] = (
                binary_df[singular] | binary_df[col]
            ).astype(int)
            # xóa cột số nhiều
            binary_df.drop(columns=[col], inplace=True)
            # cập nhật lại danh sách cột để tránh lặp vô hạn
            cols.remove(col)

# 7. Bộ STOPWORDS để loại bỏ các cột không mong muốn
STOPWORDS = {
    'fillet', 'skin', 'scaled', 'pin', 'bone', 'pin bone', 'skin on', 'trimmed', 'roes', 'cubed', 'cure', 'cured',
    'attached', 'from', 'sustainable', 'sources', 'scrubbed', 'debearded', 'warmed', 'dice', 'mixed greens',
    'beaten', 'softened', 'roughly', 'for', 'frying', 'greasing', 'plus', 'extra', 'offal', 'off',
    'ideas', 'decorating', 'of', 'a', 'the', 'to', 'in', 'and', 'on', 'one', 'two', 'dish','petals','pounded','unrolled','wrappers',
    'three', 'half', 'cup', 'cups', 'tbsp', 'tsp', 'grams', 'gram', 'ounce', 'ounces','root',
    'g', 'kg', 'ml', 'l', 'dash', 'pinch', 'handful', 'stick', 'sticks', 'opened', 'trimming',
    'optional', 'out', 'own', 'possible', 'pressed', 'prepared', 'reserved', 'required', 'ribs',
    'selected', 'separated', 'similar', 'size', 'sized', 'square', 'stand', 'style',
    'served', 'tach', 'them', 'this', 'up', 'want', 'washed', 'with', 'you', 'mix', 'rossa'
    'above', 'afford', 'alternative', 'amount', 'available', 'below', 'both', 'best',
    'bag', 'ball', 'bar', 'board', 'boat', 'can', 'case', 'choice', 'clean', 'cleaned',
    'conference', 'cooked', 'cooking', 'cooled', 'copped', 'description', 'diameter',
    'each', 'eat', 'else', 'end', 'equipment', 'extract', 'intact', 'list', 'note',
    'chopped', 'chunks', 'cube', 'diced', 'drained', 'dry', 'dusting', 'grated',
    'grating', 'halved', 'halves', 'leaf', 'leafy', 'leaves', 'picked', 'peeled', 'removed',
    'sheets', 'soda', 'serve', 'source', 'tongs', 'tip', 'total', 'piece', 'cob', 'loin',
    'heart', 'fishmonger', 'sliced', 'solids', 'pod', 'seeds', 'all', 'angle', 'baking',
    'base', 'beancurd', 'bird', 'bark', 'bowl', 'brush', 'blanched', 'blanco', 'blind',
    'briquettes', 'bought', 'branches', 'brand', 'canes', 'canned', 'carton', 'cold',
    'colour', 'crumbled', 'crust', 'dinner', 'dust', 'essence', 'finished', 'gloves',
    'grocery', 'knife', 'kitchen', 'leftovers', 'marmite', 'mixer', 'molasses', 'plastic',
    'plates', 'saucepan', 'sauce', 'pan', 'pans', 'kettle', 'miso', 'bianco', 'boned',
    'boneless', 'bonnet', 'bouillon', 'breadcumbs', 'breast', 'broth', 'brown',
    'brushing', 'butcher', 'butterflied', 'charcoal', 'chilled', 'classico', 'coating',
    'colander', 'collar', 'colouring', 'decorate', 'dried', 'drizzling', 'fat', 'pot',
    'quartered', 'recipe', 'santo', 'shaving', 'shells', 'shredded', 'shoots', 'skewer',
    'slices', 'spatula', 'sprinkling', 'stalks', 'string', 'strips', 'stuffing', 'taste',
    'thick', 'threads', 'toasted', 'tubes', 'wedges', 'whole', 'wrap', 'destoned',
    'filling', 'flakes', 'gutted', 'icing', 'leg', 'lengthways', 'needed', 'ones',
    'probe', 'roll', 'stoned', 'top', 'topping', 'platter', 'nero', 'crushed', 'pinboned', 'pork chops',
    'stock', 'altogether', 'content', 'desired', 'discarded', 'dropdown', 'ends', 'mash', 'mashed',
    'favourite', 'fine', 'finish', 'flavour', 'flavouring', 'flowers', 'four', 'free',
    'fresh', 'garnishes', 'garnish', 'glass', 'head', 'heated', 'homemade', 'ingredients', 'lamb chops', 'racks of lamb',
    'instructions', 'jar', 'jug', 'kept', 'large', 'larger', 'machine', 'mixture',
    'opener', 'paper', 'pens', 'person', 'picture', 'portion', 'preference', 'press', 'boar shoulder', 'pork belly',
    'processor', 'server', 'serving', 'shapes', 'slivers', 'small', 'spoon', 'stores',
    'supermarkets', 'tablespoons', 'teaspoons', 'temperature', 'testing', 'together','undyed','wara',
    'tray', 'trivet', 'untied', 'untrimmed', 'version', 'weight', 'whip', 'whisk',
    'batter', 'box', 'cuppa', 'dip', 'dome', 'dough', 'frosting', 'hot', 'ice', 'josh',
    'mayo', 'meal', 'meaty', 'medium', 'melted', 'nice', 'over', 'oz', 'pack', 'powder','laminating', 'hickory',
    'rippling', 'rounds', 'rub', 'skillet', 'smoke', 'smoothie', 'snaps', 'snipped','oro','matchsticks'
    'sponge', 'spread', 'starter', 'teabag', 'about', 'blue', 'cook time', 'difficulty',
    'dish name', 'fingers', 'get', 'jerry', 'jointed', 'juice', 'length', 'like', 'method','scottish','vine', 'above',
    'organic', 'portioned', 'pud', 'roast', 'scored', 'seasoning', 'split', 'strands',
    'thirds', 'torn', 'well', 'calories', 'carbs', 'fibre', 'main ingredient', 'protein',
    'saturates','english', 'pitted', 'podded', 'rinsed', 'shelled', 'brine', 'crumb', 'offcuts', 'paste', 'salad',
    'spice', 'stale', 'starch', 'concentrate', 'crumble', 'gratin', 'marinade', 'meat',
    'nests', 'parcels', 'pasty', 'pudding', 'sandwich', 'skewers', 'tied', 'veg', 'vegetables', 'roaster', 'salty', 'shoulder'
}

# Chỉ giữ lại những cột tên hợp lệ (chỉ có chữ a-z và space, không nằm trong STOPWORDS, và có ít nhất 1 giá trị 1)
valid_cols = [
    c for c in binary_df.columns
    if re.match(r'^[a-z ]+$', c) and c not in STOPWORDS and binary_df[c].sum() >= 1
]
binary_df_filtered = binary_df[valid_cols].copy()

# 8. Ghép binary_df_filtered với df gốc (loại bỏ Ingredients_list ban đầu)
result = pd.concat([df.drop(columns=['Ingredients_list']), binary_df_filtered], axis=1)

# 9. Xuất file mới
result.to_csv('covereal.csv', index=False)


### Hàm mới này 
